<a href="https://colab.research.google.com/github/KAIET98/prevision-dia-tiempo-real/blob/master/2021_01_22_Respondiendo_preguntas_Videojuegos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Respondiendo preguntas sobre los datos

Enrique Onieva Caracuel - Universidad de Deusto

https://www.linkedin.com/in/enriqueonieva/

## Instalación de librerías

In [ ]:
packages = c("tidyverse", "ggrepel", "corrplot","fastDummies","arules","arulesViz")
newpack  = packages[!(packages %in% installed.packages()[,"Package"])]

if(length(newpack)) install.packages(newpack)
a=lapply(packages, library, character.only=TRUE)

(Previamente, es necesario subir el fichero vgsales.csv)

In [ ]:
datos = read.csv("vgsales.csv")
knitr::kable(head(datos))
dim(datos)

La function de "kable" te muestra la tabla de head(datos) de una forma de "Terminal" del ordenador

---



In [ ]:
datos = datos %>% filter(Year >= 2000)
knitr::kable(head(datos,10))

In [ ]:
knitr::kable(datos %>% filter(Name == "Minecraft"))

## Juegos con más ventas en (...)
- Arrange
- top_n

In [ ]:
knitr::kable(head( datos %>% 
   arrange(desc(Global_Sales)) %>%
   top_n(3)
   )
   )

Ordenamos la zona de Other_Sales y ordenamos la tabla por medio de las ventas de Other_Sales, y visualizamos el top_10


In [ ]:
knitr::kable(head(
  datos %>%
  arrange(
    desc(
      Other_Sales
    )
  )
)
)

## Juegos con más ventas en (...) para una determinada plataforma, año...
- filter
- arrange
- top_n

In [ ]:
respuesta = datos %>% 
   filter(Year == 2008) %>%
   arrange(desc(Global_Sales)) %>%
   top_n(3)

knitr::kable(head(respuesta))

In [ ]:
# Ordenamos la tabla de datos para que me saque por WII en Estados Unidos cuales han sido los videojuegos más vendidos
head(
  datos %>%
filter(Platform == "Wii") %>%
arrange(desc(
  NA_Sales
)
)
)

## Plataforma con mayores ventas
- group_by
- summarise


In [ ]:
respuesta = datos %>% 
   group_by(Platform) %>%
   summarise(Total = sum(Global_Sales)) %>%
   arrange(desc(Total))

knitr::kable(head(respuesta,3))

In [ ]:
ggplot(respuesta, aes(x = reorder(Platform, Total), y = Total)) + geom_col()

In [ ]:

datos %>% 
group_by(Platform) %>%
summarise(Total_ventas = round(mean(Global_Sales), 2),
desv_estand = round(sd(Global_Sales), 2)
) %>% arrange(
  desc(
    Total_ventas
  )
) %>% ggplot(
  .,
  aes(Platform,
  Total_ventas, fill = Total_ventas)
) +
geom_col() +
theme_minimal() +
xlab("Plataforma")+
ylab("Ventas globales")

In [ ]:
datos%>%
group_by(Year) %>%
summarise(
  Total_Ventas = round(sum(Global_Sales)/sum(datos$Global_Sales), 
  2), 
  promedio = n()) %>% ggplot(
    .,
    aes(
      Year, Total_Ventas 
    )
  ) +
  geom_point(
    aes(fill = Year)
   ) +
  geom_jitter() +
  geom_line() +
  geom_text(aes(label=Year)
  ) +
  theme_minimal()
  





##Plataforma con más ventas por año, género...
- filter
- group_by
- summarise
- arrange

In [ ]:
print(unique(datos$Genre))
respuesta = datos %>% 
   filter(Genre == "Puzzle") %>%
   group_by(Platform) %>%
   summarise(Total = sum(Global_Sales)) %>%
   arrange(desc(Total))

knitr::kable(head(respuesta,3))

In [ ]:
head(
  datos %>%
  filter(
    Genre == "Sports"
  ) %>%
  group_by(
    Platform
  ) %>%
  summarise(
    Total_ventas = sum(Global_Sales)
  ) %>%
  arrange(desc(Total_ventas))
) 

## Ventas consola y año

In [ ]:
datos2 = datos %>%
  group_by(Platform, Genre, Year) %>%
  summarise(ventas = sum(Global_Sales)) %>%
  arrange(desc(ventas, Platform, Genre ), Year)

knitr::kable(head(datos2,30))




Plataforma con más ventas en un año, o mejor año para una plataforma

In [ ]:
respuesta = datos2 %>%
    filter(Year == 2010) %>%
    arrange(desc(ventas))

knitr::kable(head(respuesta,3))

In [ ]:
respuesta = datos2 %>%
    filter(Platform == "PS3") %>%
    arrange(desc(ventas))

knitr::kable(head(respuesta,3))

## Necesitamos generar nueva información para responder a otras preguntas
- Juegos que se venden mucho más en EEUU que EUROPA

In [ ]:
datos2 = datos %>%
  group_by(Name) %>%
  summarise(EEUU = sum(NA_Sales),
            EUROPA = sum(EU_Sales)) %>%
  filter(EEUU>1) %>%
  filter(EUROPA>1) %>%
  mutate(Proporcion = EEUU/EUROPA) %>%
  arrange(Proporcion)
  
knitr::kable(head(datos2,3))
knitr::kable(tail(datos2,3))

In [ ]:
datos3 = rbind(head(datos2,10),tail(datos2,10))
ggplot(datos3, aes(x = EEUU, y = EUROPA, col = Proporcion)) + 
   geom_point() + 
   geom_label_repel(aes(label = Name), max.overlaps = 100)

## Género y lugar
- pivot

In [ ]:
datos2 = datos %>%
 group_by(Genre) %>%
  summarise(EEUU = sum(NA_Sales),
            EUROPA = sum(EU_Sales),
            JAPON = sum(JP_Sales),
            OTRO = sum(Other_Sales),
            GLOBAL = sum(Global_Sales)) %>%
  mutate(EEUU = 100*EEUU / GLOBAL) %>%
  mutate(EUROPA = 100*EUROPA / GLOBAL) %>%
  mutate(JAPON = 100*JAPON / GLOBAL) %>%
  mutate(OTRO = 100*OTRO / GLOBAL) %>%
  select(Genre, EEUU, EUROPA, JAPON, OTRO)

knitr::kable((datos2))

In [ ]:
datos3 = datos2 %>%
  pivot_longer(!Genre, names_to = "Region", values_to = "perc")

knitr::kable(head(datos3))


In [ ]:
ggplot(datos3, aes(x = Genre, y = perc, fill = Genre)) + 
  geom_col(position = "dodge") +
  coord_flip()+ 
  facet_wrap(~Region) + 
  labs(x = "", y = "") + 
  theme_minimal()

## Relación entre columnas

In [ ]:
resultado = round(cor(datos %>% select(ends_with("Sales"))),2)
knitr::kable(resultado)

In [ ]:
corrplot.mixed(resultado)

In [ ]:
resultado = table(datos$Genre, datos$Platform)
knitr::kable(resultado)

In [ ]:
resultado = prop.table(table(datos$Genre, datos$Platform), margin = 1)
resultado = round(100*resultado,1)
knitr::kable(resultado)

In [ ]:
resultado = prop.table(table(datos$Genre, datos$Platform), margin = 2)
resultado = round(100*resultado,1)
knitr::kable(resultado)

In [ ]:
datos2 = datos %>% 
  group_by(Genre, Platform) %>%
  summarise(n = n()) %>%
  group_by(Platform) %>%
  mutate(p = n/sum(n)) %>%
  filter(p>0.1)

knitr::kable(head(datos2))
ggplot(datos2, aes(x = Platform, y = Genre, fill = p)) + 
  geom_tile()

## Clustering

In [ ]:
datos2 = datos %>%
  select(ends_with("Sales"))%>%
  mutate(NA_Sales = NA_Sales/Global_Sales,
         EU_Sales = EU_Sales/Global_Sales,
         JP_Sales = JP_Sales/Global_Sales,
         Other_Sales = Other_Sales/Global_Sales) %>%
  select(-Global_Sales)

datos2 = round(100*datos2)
knitr::kable(head(datos2))

In [ ]:
distances = dist(datos2, method = "euclidean")
clustering = hclust(distances, method="ward.D2")
plot(clustering, labels = FALSE, hang = -1)
rect.hclust(clustering, k = 6, border = 2:10) 

In [ ]:
miembros = cutree(clustering, k = 6)
print(head(miembros,100))

datos3 = datos2 %>%
  mutate(Name = datos$Name, 
         cluster = miembros,
         ventas = datos$Global_Sales)

knitr::kable(head(datos3))

In [ ]:
centroides = datos3 %>%
          select(-Name) %>%
          group_by(cluster) %>%
          summarise_all(mean) %>% # los juegos del cluster 1 en promedio hacen el 46% de las ventas en N.A
          summarise_all(round)

knitr::kable(centroides)

In [ ]:
table(miembros)
resultado = prop.table(table(miembros,datos$Genre), margin = 1)
resultado = round(100*resultado,1)
knitr::kable(resultado)

In [ ]:
resultado = datos3 %>%
  group_by(cluster) %>%
  arrange(desc(ventas)) %>%
  top_n(2) %>%
  arrange(cluster)

knitr::kable(resultado)

## Reglas de asociación

In [ ]:
datos2 = datos %>%
  dummy_cols(select_columns = "Platform") %>%
  select(-Rank,-Platform,-Global_Sales) %>%
  group_by(Name, Genre, Publisher,Year) %>%
  summarise_all(sum)

knitr::kable(head(datos2,15))

In [ ]:
datos3 = datos2 %>%
  ungroup() %>%
  select(-Name) %>%
  mutate(Year = case_when(Year < 2010 ~ "2000s",TRUE ~ "2010s")) %>%
  mutate_at(vars(ends_with("Sales")),function(x){ifelse(x<mean(x),NA,"high")}) %>%
  mutate_at(vars(starts_with("Platform")),function(x){ifelse(x==0,NA,"YES")})

knitr::kable(head(datos3,15))

In [ ]:
class(datos3)
datos.trans = as(data.frame(datos3), "transactions")
summary(datos.trans)


#Densidad: estos metodos necesitan transacciones o compras, tu tienes diferentes clientes o tickets, y tienes a la vez diferentes productos. Pues, lo que pasa que en cualquier negocio, tu vas a tener muchos tickets porque mucha gente viene a comprar, y tienes muchos productos. Por lo que tienes una matriz bianaria, donde en el primer ticket esta el primer producot, no está el segundo esta el tercero y demas. Por lo que una matriz tiene muchas columans. Una persona, sola, compra un porcentaje muy bajo d elos productos. En la compra que tu haces, estarían todos los productos del supermercado pero con solamente unos seleccinoados. En este caso están el 0.01037836 , es decir el 1,07% de los productos.

In [ ]:
rules = apriori(datos.trans, parameter = list(supp = 0.05, 
# dame solo reglas que afecten como minimo al 0.05 d elos juegos
conf = 0.9,# dame reglas con mucha certeza
 minlen = 2, # minimo reglas que asocien 2 productos
 maxlen=3 # maximo que afecten a 3, no quiero una relag muy confusa
 ))


#LHS: LEFT HANDS SIDE, 
#RHS: RIGHT HAND SIDE
# count: cuantos lo cumplen 
#support: cuantos individuos cumplen la regla entre los totales.
# coverage: cobertura, es simimlar al soporte, lo unico que considera el LHS.
#lift: es una medida que nos da una idea de como de sorprendente es la regla, en funcion a la parte que tienen en su consequente, es decir RHS. Cuanto más alto sea el Lift, es decir, cuanto más 
knitr::kable(DATAFRAME(sort(rules, by="confidence", decreasing=TRUE)))

In [ ]:
rules = apriori(datos.trans, parameter = list(supp = 0.01, conf = 0.5, maxlen=2))
rules = subset(rules, subset = (rhs %in% c("NA_Sales=high","EU_Sales=high","JP_Sales=high","Other_Sales=high")))
knitr::kable(DATAFRAME(sort(rules, by="confidence", decreasing=TRUE)))

In [ ]:
plot(rules, method = "graph", control=list(cex=1))